In [1]:
from PIL import Image, ImageOps
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import SGDClassifier

In [2]:
def aanmakenDf (groep, label):
#Drie lege lijsten worden aangemaakt die dan uiteindelijk samen dataframe zullen vormen 
    Pixels= []
    Label =[]
  
#fotos inlezen en omzetten per groep. 
    for foto in os.listdir(groep):
        beeld = Image.open(groep+foto)
#fotos omzetten in grijswaarde of 2 dimensionele array ipv 3x2
        beeld_grijs = ImageOps.grayscale(beeld)

#fotos resizen voor leesbaarheid tijdens testen, en eventueel voor latere experimenten
        beeld_formaat = beeld_grijs.resize((beeld_grijs.width // 1, beeld_grijs.height //1 ))       
        
#fotos omzetten naar numpy array
        data_2dim = np.asarray(beeld_formaat, dtype=np.uint8)
        data = data_2dim.flatten()

#3 lijsten opvullen, eerste met data van foto, tweede met label en derde met naam van foto. 
#De lengte van elke lijst = aantal fotos in bijhorende map
        Pixels.append(data)
        Label.append(label)

#Per groep worden de drie aangemaakte lijsten omgezet naar panda df
    dict = {'data': Pixels, 'label': Label}
    df = pd.DataFrame(dict)
    return df

In [3]:
#Dataframe aanmaken
aanwezig = r"classificatie/aanwezig/"  #852 foto's 
buiten = r"classificatie/buiten/"       #389 foto's
niets = r"classificatie/niets/"         #1399 foto's
#Eerst wordt per map een panda dataframe aangemaakt
nietsDf = aanmakenDf(niets, 0)
aanwezigDf = aanmakenDf(aanwezig, 1)
buitenDf = aanmakenDf(buiten, 2)
#vervolgens worden ze alle drie samengevoegd tot 1 groot dataframe
volledigDataframe = pd.concat([nietsDf, aanwezigDf, buitenDf], ignore_index=True)

In [4]:
#Opsplitsen in gestratificieerde testset en trainingsset
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(volledigDataframe, volledigDataframe['label']):
    strat_train_set = volledigDataframe.loc[train_index]
    strat_test_set = volledigDataframe.loc[test_index] 
#X = data, y = label. Dit voor trainingsset en testset  
X_train, X_test, y_train, y_test = strat_train_set['data'], strat_test_set['data'], strat_train_set['label'], strat_test_set['label']

In [5]:
#Omzetten naar np.array
X_train_array = np.array(X_train.tolist())
X_test_array = np.array(X_test.tolist())
y_train_array = np.array(y_train)
y_test_array = np.array(y_test)


In [6]:
# scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_array.astype(np.float64))
X_test_scaled = scaler.fit_transform(X_test_array.astype(np.float64))

In [7]:
X_test_scaled.size

118947840

In [17]:
#RandomforestClassifier
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
#forest_clf.fit(X_train_array, y_train_array)
forest_clf.fit(X_train_scaled, y_train_array)

RandomForestClassifier(random_state=42)

In [18]:
forest_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [19]:
param_grid = { 
    'n_estimators': [5, 10, 100, 300, 500, 800],  #controleert het aantal bomen in de classifier ; standaard 100
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [None,5,8, 20],  #Bepaalt de hoogte tot waar een boom kan groeien, één van de belangrijkste parameters naar accuracy toe.
                                #Let op: Niet hoe meer, hoe beter, maar uitkijken voor overfitting. Standaard: none, wat betekent dat ze blijven groeien tot bladeren puur zijn
    'min_samples_split' : [2,5,20,50,200],
    'min_samples_leaf': [1,100,300,500,800],
    'max_leaf_nodes' : [None,5,8, 20,50],  #Helpt om max depth te beperken en dus om overfittig tegen te gaan. 
}

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
CV_forest_clf = RandomizedSearchCV(forest_clf, param_grid, n_iter=1000, cv=3, verbose = 2)
#CV_forest_clf.fit(X_train_array, y_train_array)
CV_forest_clf.fit(X_train_scaled, y_train_array)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits
[CV] END max_depth=5, max_features=log2, max_leaf_nodes=50, min_samples_leaf=300, min_samples_split=20, n_estimators=500; total time=   3.9s
[CV] END max_depth=5, max_features=log2, max_leaf_nodes=50, min_samples_leaf=300, min_samples_split=20, n_estimators=500; total time=   3.7s
[CV] END max_depth=5, max_features=log2, max_leaf_nodes=50, min_samples_leaf=300, min_samples_split=20, n_estimators=500; total time=   4.0s
[CV] END max_depth=5, max_features=log2, max_leaf_nodes=8, min_samples_leaf=500, min_samples_split=50, n_estimators=100; total time=   2.2s
[CV] END max_depth=5, max_features=log2, max_leaf_nodes=8, min_samples_leaf=500, min_samples_split=50, n_estimators=100; total time=   2.2s
[CV] END max_depth=5, max_features=log2, max_leaf_nodes=8, min_samples_leaf=500, min_samples_split=50, n_estimators=100; total time=   2.2s
[CV] END max_depth=None, max_features=sqrt, max_leaf_nodes=5, min_samples_leaf=800, min_samp

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=1000,
                   param_distributions={'max_depth': [None, 5, 8, 20],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'max_leaf_nodes': [None, 5, 8, 20, 50],
                                        'min_samples_leaf': [1, 100, 300, 500,
                                                             800],
                                        'min_samples_split': [2, 5, 20, 50,
                                                              200],
                                        'n_estimators': [5, 10, 100, 300, 500,
                                                         800]},
                   verbose=2)

In [21]:
CV_forest_clf.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': 50,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [22]:
#Beste waarden op basis van non -scaled
#best_forest_clf = RandomForestClassifier(random_state=42, min_samples_split =  20,n_estimators= 500)

#Beste waarden op basis van scaled
best_forest_clf = RandomForestClassifier(random_state=42, min_samples_split =  2,n_estimators= 300, max_leaf_nodes = 50)

In [25]:
#Accuraatheid van random forest, oneVSone, supportVectorMachine en voting
from sklearn.metrics import accuracy_score
for clf in (best_forest_clf, forest_clf):
 #clf.fit(X_train_array, y_train_array)
 clf.fit(X_train_scaled , y_train_array)
 y_pred = clf.predict(X_test_scaled)
 print(clf.__class__.__name__, accuracy_score(y_test_array, y_pred))

RandomForestClassifier 0.9488636363636364
RandomForestClassifier 0.9431818181818182


In [26]:
parameters = { 
    'n_estimators': [300,400,500],  #controleert het aantal bomen in de classifier ; standaard 100
    'min_samples_split' : [1,2,3,4,5,20],
    'max_leaf_nodes' : [50,60,70,100],  #Helpt om max depth te beperken en dus om overfittig tegen te gaan. 
}

In [27]:
from sklearn.model_selection import GridSearchCV
grid_forest_clf = GridSearchCV(forest_clf,parameters,cv=5, verbose=2)
grid_forest_clf.fit(X_train_scaled, y_train_array)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=300; total time=   2.3s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=300; total time=   2.1s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=300; total time=   1.8s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=300; total time=   1.9s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=300; total time=   1.7s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=400; total time=   1.8s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=400; total time=   1.7s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=400; total time=   1.7s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=400; total time=   1.8s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=400; total time=   1.8s
[CV] END max_leaf_nodes=50, min_samples_split=1, n_estimators=500; total time=

C:\Users\berto\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\berto\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\berto\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\berto\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\parallel.py

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [50, 60, 70, 100],
                         'min_samples_split': [1, 2, 3, 4, 5, 20],
                         'n_estimators': [300, 400, 500]},
             verbose=2)

In [28]:
grid_forest_clf.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': 50,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [8]:
from sklearn.ensemble import RandomForestClassifier
allerbeste_forest_clf = RandomForestClassifier(random_state=42, min_samples_split = 2,n_estimators= 300, max_leaf_nodes = 50)

In [9]:
#Accuraatheid van random forest, oneVSone, supportVectorMachine en voting
from sklearn.metrics import accuracy_score
allerbeste_forest_clf.fit(X_train_scaled , y_train_array)
y_pred = allerbeste_forest_clf.predict(X_test_scaled)
print(allerbeste_forest_clf.__class__.__name__, accuracy_score(y_test_array, y_pred))

RandomForestClassifier 0.9488636363636364


In [32]:
#Confusion matrix voor forestClassifier
#rijen zijn gegeven klasses, kolommen voorspelde
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_train_forestPred = cross_val_predict(best_forest_clf, X_train_scaled, y_train_array, cv=3)
conf_mx = confusion_matrix(y_train_array, y_train_forestPred)
conf_mx

array([[1109,   10,    0],
       [  44,  606,   32],
       [   3,   35,  273]], dtype=int64)

In [ ]:
#Precision en Recall
from sklearn.metrics import precision_score, recall_score
precision_score(y_train_array, y_train_forestPred) 

In [ ]:
recall_score(y_train_array, y_train_forestPred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train_array, y_train_forestPred)

In [ ]:
from sklearn.metrics import precision_recall_curve
y_train_forestPred = cross_val_predict(best_forest_clf, X_train_scaled, y_train_array, cv=3,
method="decision_function")
precisions, recalls, thresholds = precision_recall_curve(y_train_array, y_train_forestPred)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
  plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
  plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
  
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

In [ ]:
#Kiezen van mate van precisie. Hier kan mee worden gespeeld.
#Op basis hiervan bepalen hoeveel foto's uit analyse worden verwijderd...???
threshold_98_precision = thresholds[np.argmax(precisions >= 0.98)]
threshold_98_precision

In [ ]:
y_train_pred_98 = (y_train_forestPred >= threshold_98_precision)
precision_score(y_train_array, y_train_pred_98)

In [ ]:
recall_score(y_train_array, y_train_pred_98)

In [ ]:
#f1 score komt volgens mij overeen met accuraatheid. Deze algemene score kan dalen ten voordele van stijgende recall
#Op  die manier merk je zeker een kat op die naar buiten gaat, net zoals de dokter de hersentumor.
#Er zullen idd ook meer valse alarmen zijn, maar die zijn dan duidelijk te zien op de foto, dus minder erg???
f1_score(y_train_array, y_train_pred_98)

In [ ]:
#Treshold is volgens mij in te voeren adhv predicted_proba
#Dit is echter moeilijk bij multiclass random forest. Ik heb hiervoor geen opl gevonden, wel gewicht aan klasse geven...
#https://stackoverflow.com/questions/49785904/how-to-set-threshold-to-scikit-learn-random-forest-model
#https://stackoverflow.com/questions/57307095/change-threshold-value-for-random-forest-classifier
#https://stackoverflow.com/questions/70813656/multiclass-classification-with-random-forest-how-to-increase-recall-instead-of

In [10]:
#class weight toevoegen aan model. In ons geval meer gewicht aan klasse 2!
# https://stackoverflow.com/questions/58275113/proper-use-of-class-weight-parameter-in-random-forest-classifier
#Berekening class weight: Totaal cases / aantal klasses * cases in klasse
allerallerbeste_forest_clf = RandomForestClassifier(random_state=42, min_samples_split = 2,n_estimators= 300, max_leaf_nodes = 50, class_weight = {0: 0.62, 1: 10.16, 2: 20.25})

In [11]:
#Accuraatheid van random forest, oneVSone, supportVectorMachine en voting
from sklearn.metrics import accuracy_score
allerallerbeste_forest_clf.fit(X_train_scaled , y_train_array)
y_pred = allerallerbeste_forest_clf.predict(X_test_scaled)
print(allerallerbeste_forest_clf.__class__.__name__, accuracy_score(y_test_array, y_pred))

RandomForestClassifier 0.7840909090909091


In [49]:
#Confusion matrix voor forestClassifier
#rijen zijn gegeven klasses, kolommen voorspelde
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_train_forestPred = cross_val_predict(allerallerbeste_forest_clf, X_train_scaled, y_train_array, cv=3)
conf_mx = confusion_matrix(y_train_array, y_train_forestPred)
conf_mx

array([[780, 339,   0],
       [ 65, 578,  39],
       [  2,  28, 281]], dtype=int64)